In [9]:
import numpy as np
from scipy.special import expit
import time
import math

with open("X_train.txt", 'r') as train_data:
    lines_train_data = train_data.readlines()
with open("y_train.txt", 'r') as train_label:
    lines_train_label = train_label.readlines()
    
with open("X_test.txt", 'r') as test_data:
    lines_test_data = test_data.readlines()
with open("y_test.txt", 'r') as test_label:
    lines_test_label = test_label.readlines()

print(len(lines_train_data))
print(len(lines_train_label))
print(len(lines_test_data))
print(len(lines_test_label))
#print(lines_train_data[0])


7352
7352
2947
2947


In [10]:
train_data_array = np.zeros((len(lines_train_data), 561))        # Assuming 561 variables per pattern

for i, line in enumerate(lines_train_data):

    values = [float(val) for val in line.strip().split()]
    train_data_array[i, :] = values

#print(train_data_array)

train_label_array = np.zeros((len(lines_train_label), 1))       # Assuming 561 variables per pattern

for i in range(len(lines_train_label)):

    train_label_array[i] = lines_train_label[i]

    
test_data_array = np.zeros((len(lines_test_data), 561))        # Assuming 561 variables per pattern

for k, linee in enumerate(lines_test_data):

    values = [float(val) for val in linee.strip().split()]
    test_data_array[k, :] = values


test_label_array = np.zeros((len(lines_test_label), 1))         # Assuming 561 variables per pattern

for g in range(len(lines_test_label)):

    test_label_array[g] = lines_test_label[g]    
    
#print(test_label_array)

In [11]:
print(train_data_array.shape)
print(train_label_array.shape)
print(test_data_array.shape)
print(test_label_array.shape)

(7352, 561)
(7352, 1)
(2947, 561)
(2947, 1)


In [12]:
per = np.random.permutation(train_data_array.shape[0])
shuffled_train_data_array = train_data_array
shuffled_train_label_array = train_label_array
for old, new in enumerate(per):
    shuffled_train_data_array[new,:] = train_data_array[old,:]
    shuffled_train_label_array[new,:] = train_label_array[old,:]

In [13]:
class HAR:
    
    def initilaze(self, hidden1_size, hidden2_size, learning_rate, batch_size, epochs, momentum):
        
        self.IH1_weight = np.random.uniform(-0.01, 0.01, size=(561 ,hidden1_size))
        self.H1H2_weight = np.random.uniform(-0.01, 0.01, size=(hidden1_size, hidden2_size))
        self.H2O_weight = np.random.uniform(-0.01, 0.01, size=(hidden2_size, 6))

        self.learning_rate = learning_rate
        self.hidden1_size = hidden1_size
        self.hidden2_size = hidden2_size
        self.batch_size = batch_size
        self.epochs = epochs
        self.momentum = momentum
        return    
    
    def SOFTMAX(self,v_1):
        
        e_v_1 = np.exp(v_1)
        all_prob = e_v_1.sum()
        result_matrix = np.round(e_v_1/all_prob, 5)
        
        return result_matrix
    
    
    def RELU_derivative(self,x):
        return np.where(x < 0, 0, 1)   
    
    
    def RELU(self,x):
        return np.maximum(0,x) 

    def forward_one_data(self,one_row_data, label):
        
        self.label = int(label)
        self.reshaped_data = one_row_data.reshape(1,561)               #(1,561)
        #print(self.reshaped_data.shape)
        self.v_1 = np.dot(self.reshaped_data, self.IH1_weight)      #(1,hidden1_size)
        #print(self.v_1.shape)
        self.y_1 = self.RELU(self.v_1)                         #(1,hidden1_size)
        self.v_2 = np.dot(self.y_1, self.H1H2_weight)          #(1,hidden2_size)
        self.y_2 = self.RELU(self.v_2)                         #(1,hidden2_size)
        #print(self.y_2.shape)
        self.v_3 = np.dot(self.y_2, self.H2O_weight)           #(1,6)
        #print(self.v_3.shape)
        self.y_3 = self.SOFTMAX(self.v_3)                      #(1,6)
        #print(self.y_3.shape)                      
        self.output = self.y_3                                 #(1,6)
        #print(self.output.shape)
        self.error = np.log(self.output[0, int(label) - 1] + 0.000000000000001)
        
        return self.error, self.output
    
    def backward_propagation(self):
        
        row = - self.output
        row[0, self.label-1] = 1 - self.output[0, self.label-1]     #(1,6)
        #print(row.shape)
        update_H2O = self.learning_rate * np.dot(self.y_2.T, row)   # (hidden2_size, 1) @ (1,6) = (hidden2_size, 6)
        
        der2 = self.RELU_derivative(self.v_2)                          # (1,hidden2_size)
        diag_der2 = np.diag(der2[0])                                   # (hidden2_size,hidden2_size)
        row_2 = np.dot(np.dot(row, self.H2O_weight.T), diag_der2)      # (1, 6) @ (6, h2) @ (h2, h2) = (1, h2)
        update_H1H2 = self.learning_rate * np.dot(self.y_1.T, row_2)   # (h1, 1) @ (1, h2) = (h1, h2)
        
        der3 = self.RELU_derivative(self.v_1)                                # (1,hidden1_size)
        diag_der3 = np.diag(der3[0])                                         # (hidden1_size, hidden1_size)
        row_3 = np.dot(np.dot(row_2,self.H1H2_weight.T), diag_der3)          # (1, h2) @ (h2, h1) @ (h1, h1) = (1, h1)
        update_IH1 = self.learning_rate * np.dot(self.reshaped_data.T, row_3)# (561, 1) @ (1, h1) = (561, h1)

        return update_H2O, update_H1H2, update_IH1                                          

    
    def update_weights(self,c_up_H2O, c_up_H1H2, c_up_IH1):

        self.H2O_weight += c_up_H2O
        self.H1H2_weight += c_up_H1H2
        self.IH1_weight += c_up_IH1
    
    
    def train(self,full_train_data, full_train_label):
        
        self.prev2_val_error = 9999999999
        self.prev1_val_error = 9999999999
        self.val_error = 0
        
        for epoch in range(self.epochs):
            batch_counter = 0
            self.one_epoch_error = 0
            cum_up_H2O = 0
            cum_up_H1H2 = 0
            cum_up_IH1 = 0
            for data_index in range(int(full_train_data.shape[0]*0.9)):
                
                for_error, out = self.forward_one_data(full_train_data[data_index], full_train_label[data_index])
                self.one_epoch_error += for_error
                up_H2O, up_H1H2, up_IH1  = self.backward_propagation()
                
                cum_up_H2O += up_H2O
                cum_up_H1H2 += up_H1H2
                cum_up_IH1 += up_IH1
                
                batch_counter +=1
        
                if (self.batch_size > 0 and batch_counter == self.batch_size ) or self.batch_size == 0:
                    self.update_weights(cum_up_H2O, cum_up_H1H2, cum_up_IH1)
                    
                    batch_counter = 0
                    
                    if self.momentum == 0.09:
                    
                        cum_up_H2O = cum_up_H2O * 0.09
                        cum_up_H1H2 = cum_up_H1H2 * 0.09
                        cum_up_IH1 = cum_up_IH1 * 0.09
                    else:
                        cum_up_H2O = cum_up_H2O * 0
                        cum_up_H1H2 = cum_up_H1H2 * 0
                        cum_up_IH1 = cum_up_IH1 * 0
                        
                
            print(f"Epoch {epoch+1} Train Error: {(-self.one_epoch_error):.7f}", end=" ")
            
            self.val_error = 0

            for data_index in range(int(full_train_data.shape[0]*0.9),full_train_data.shape[0]):
                
                for_error, out = self.forward_one_data(full_train_data[data_index], full_train_label[data_index])
                self.val_error += for_error
                
            print(f" Validation Error: {(-self.val_error):.7f}")
            
            
            if (-(self.val_error) > self.prev1_val_error) and (self.prev1_val_error > self.prev2_val_error):
                print("\nDue to Validation Error Data: Early Stoping is happened at", str(epoch+1)+". Epoch.")
                break
                 
            self.prev2_val_error = self.prev1_val_error
            self.prev1_val_error = -(self.val_error)
            self.val_error = 0 

        
    def test(self,full_test_data, full_test_label):
        acc = 0
        total_test = 0
        for data_index in range(full_test_data.shape[0]):
            total_test += 1
            for_error, out = self.forward_one_data(full_test_data[data_index], full_test_label[data_index])
            result = int(np.argmax(out))
            
            if result == (int(full_test_label[data_index] - 1)):
                acc += 1
            
        print("\nTest Results are:\n In", total_test, "number of test data\n Accurarcy = %"+ str((acc/total_test)*100))
        
        
        
        
    def missclas_counter(self,full_data,full_label):
        self.class_count_list = [0, 0, 0, 0, 0, 0]
        
        for data_index in range(full_data.shape[0]):
            
            for_error, out = self.forward_one_data(full_data[data_index], full_label[data_index])
            result = int(np.argmax(out))
            
            if result == (int(full_label[data_index] - 1)):
                a = 1
            else:
                self.class_count_list[int(full_label[data_index] - 1)] += 1
                            
        print("Miss clasification stats are:\nClass 1:"+str(self.class_count_list[0]))        
        print("Class 2:  "+str(self.class_count_list[1]))
        print("Class 3:  "+str(self.class_count_list[2]))        
        print("Class 4:  "+str(self.class_count_list[3]))        
        print("Class 5:  "+str(self.class_count_list[4]))        
        print("Class 6:  "+str(self.class_count_list[5]))

        
    def forward_one_data_dropout(self,one_row_data, label,dropout_mask):

        
        self.label = int(label)
        self.reshaped_data = one_row_data.reshape(1,561)               #(1,561)
        #print(self.reshaped_data.shape)
        self.v_1 = np.dot(self.reshaped_data, self.IH1_weight)      #(1,hidden1_size)
        #print(self.v_1.shape)
        self.y_1 = self.RELU(self.v_1)                         #(1,hidden1_size)
        self.v_2 = np.dot(self.y_1, self.H1H2_weight)          #(1,hidden2_size)
        self.y_2 = self.RELU(self.v_2)                         #(1,hidden2_size)
        #print(self.y_2.shape)
        self.v_3 = np.dot(self.y_2, self.H2O_weight)           #(1,6)
        self.v_3
        #print(self.v_3.shape)
        self.y_3 = self.SOFTMAX(self.v_3)                      #(1,6)
        #print(self.y_3.shape)                      
        self.output = self.y_3                                 #(1,6)
        #print(self.output.shape)
        self.error = np.log(self.output[0, int(label) - 1] + 0.000000000000001)
        
        return self.error, self.output 
    
    def train_dropout(self,full_train_data, full_train_label,dropout_rate):
        
        self.prev2_val_error = 9999999999
        self.prev1_val_error = 9999999999
        self.val_error = 0
        
        for epoch in range(self.epochs):
            self.dropout_mask = (np.random.rand(1, self.hidden2_size) < dropout_rate) / dropout_rate
            batch_counter = 0
            self.one_epoch_error = 0
            cum_up_H2O = 0
            cum_up_H1H2 = 0
            cum_up_IH1 = 0
            for data_index in range(int(full_train_data.shape[0]*0.9)):
                
                for_error, out = self.forward_one_data_dropout(full_train_data[data_index], full_train_label[data_index])
                self.one_epoch_error += for_error
                up_H2O, up_H1H2, up_IH1  = self.backward_propagation()
                
                cum_up_H2O += up_H2O
                cum_up_H1H2 += up_H1H2
                cum_up_IH1 += up_IH1
                
                batch_counter +=1
        
                if (self.batch_size > 0 and batch_counter == self.batch_size ) or self.batch_size == 0:
                    self.update_weights(cum_up_H2O, cum_up_H1H2, cum_up_IH1)
                    
                    batch_counter = 0
                    
                    if self.momentum == 0.9:
                    
                        cum_up_H2O = cum_up_H2O * 0.9
                        cum_up_H1H2 = cum_up_H1H2 * 0.9
                        cum_up_IH1 = cum_up_IH1 * 0.9
                    else:
                        cum_up_H2O = cum_up_H2O * 0
                        cum_up_H1H2 = cum_up_H1H2 * 0
                        cum_up_IH1 = cum_up_IH1 * 0
                        
                
            print(f"Epoch {epoch+1} Train Error: {self.one_epoch_error:.7f}", end=" ")
            
            self.val_error = 0

            for data_index in range(int(full_train_data.shape[0]*0.9),full_train_data.shape[0]):
                
                for_error, out = self.forward_one_data(full_train_data[data_index], full_train_label[data_index])
                self.val_error += for_error
                
            print(f"Validation Error: {(-self.val_error):.7f}")
            
            
            if (-(self.val_error) > self.prev1_val_error) and (self.prev1_val_error > self.prev2_val_error):
                print("\nDue to Validation Error Data: Early Stoping is happened at", str(epoch+1)+". Epoch.")
                break
                 
            self.prev2_val_error = self.prev1_val_error
            self.prev1_val_error = -(self.val_error)
            self.val_error = 0 



In [14]:
network = HAR()

In [17]:
case1 = HAR()
case2 = HAR()
case3 = HAR()
case4 = HAR()
case5 = HAR()
case6 = HAR()
case7 = HAR()
case8 = HAR()
case9 = HAR()
case10 = HAR()
case11 = HAR()
case12 = HAR()
case13 = HAR()
case14 = HAR()
case15 = HAR()
case16 = HAR()

"""
case1 = mini_batch = 0 / lear_rate = 0.001 / N2 = 100 / Momentum = 0
case2 = mini_batch = 0 / lear_rate = 0.001 / N2 = 100 Momentum = 0.9
case3 = mini_batch = 0 / lear_rate = 0.001 / N2 = 200 Momentum = 0
case4 = mini_batch = 0 / lear_rate = 0.001 / N2 = 200 Momentum = 0.9
case5 = mini_batch = 0 / lear_rate = 0.01 / N2 = 100 Momentum = 0
case6 = mini_batch = 0 / lear_rate = 0.01 / N2 = 100 Momentum = 0.9
case7 = mini_batch = 0 / lear_rate = 0.01 / N2 = 200 Momentum = 0
case8 = mini_batch = 0 / lear_rate = 0.01 / N2 = 200 Momentum = 0.9
case9 = mini_batch = 0 / lear_rate = 0.001 / N2 = 100 / Momentum = 0
case10 = mini_batch = 50 / lear_rate = 0.001 / N2 = 100 Momentum = 0.9
case11 = mini_batch = 50 / lear_rate = 0.001 / N2 = 200 Momentum = 0
case12 = mini_batch = 50 / lear_rate = 0.001 / N2 = 200 Momentum = 0.9
case13 = mini_batch = 50 / lear_rate = 0.01 / N2 = 100 Momentum = 0
case14 = mini_batch = 50 / lear_rate = 0.01 / N2 = 100 Momentum = 0.9
case15 = mini_batch = 50 / lear_rate = 0.01 / N2 = 200 Momentum = 0
case16 = mini_batch = 50 / lear_rate = 0.01 / N2 = 200 Momentum = 0.9
"""


'\ncase1 = mini_batch = 0 / lear_rate = 0.001 / N2 = 100 / Momentum = 0\ncase2 = mini_batch = 0 / lear_rate = 0.001 / N2 = 100 Momentum = 0.9\ncase3 = mini_batch = 0 / lear_rate = 0.001 / N2 = 200 Momentum = 0\ncase4 = mini_batch = 0 / lear_rate = 0.001 / N2 = 200 Momentum = 0.9\ncase5 = mini_batch = 0 / lear_rate = 0.01 / N2 = 100 Momentum = 0\ncase6 = mini_batch = 0 / lear_rate = 0.01 / N2 = 100 Momentum = 0.9\ncase7 = mini_batch = 0 / lear_rate = 0.01 / N2 = 200 Momentum = 0\ncase8 = mini_batch = 0 / lear_rate = 0.01 / N2 = 200 Momentum = 0.9\ncase9 = mini_batch = 0 / lear_rate = 0.001 / N2 = 100 / Momentum = 0\ncase10 = mini_batch = 50 / lear_rate = 0.001 / N2 = 100 Momentum = 0.9\ncase11 = mini_batch = 50 / lear_rate = 0.001 / N2 = 200 Momentum = 0\ncase12 = mini_batch = 50 / lear_rate = 0.001 / N2 = 200 Momentum = 0.9\ncase13 = mini_batch = 50 / lear_rate = 0.01 / N2 = 100 Momentum = 0\ncase14 = mini_batch = 50 / lear_rate = 0.01 / N2 = 100 Momentum = 0.9\ncase15 = mini_batch = 5

In [18]:
case1.initilaze(300, 100, 0.001, 0, 100, 0)
case1.train(shuffled_train_data_array, shuffled_train_label_array)
case1.test(test_data_array, test_label_array)
case1.missclas_counter(train_data_array, train_label_array)
case1.missclas_counter(test_data_array, test_label_array)

Epoch 1 Train Error: 11841.3889342  Validation Error: 1309.1802399
Epoch 2 Train Error: 9836.4876841  Validation Error: 819.3826753
Epoch 3 Train Error: 6199.8036952  Validation Error: 516.5009716
Epoch 4 Train Error: 4525.2020879  Validation Error: 413.1786433
Epoch 5 Train Error: 3831.7790099  Validation Error: 348.7590071
Epoch 6 Train Error: 3376.6312377  Validation Error: 304.9189845
Epoch 7 Train Error: 3067.1795700  Validation Error: 272.0750236
Epoch 8 Train Error: 2789.4172218  Validation Error: 239.9743715
Epoch 9 Train Error: 2332.1099201  Validation Error: 180.6723479
Epoch 10 Train Error: 1607.9861048  Validation Error: 120.1586384
Epoch 11 Train Error: 1242.2379371  Validation Error: 95.4158166
Epoch 12 Train Error: 1079.5137500  Validation Error: 83.2584895
Epoch 13 Train Error: 978.2508895  Validation Error: 75.7677750
Epoch 14 Train Error: 902.4594767  Validation Error: 70.6381676
Epoch 15 Train Error: 841.8083426  Validation Error: 66.6060844
Epoch 16 Train Error: 790

In [19]:
case2.initilaze(300, 100, 0.001, 0, 100, 0.9)
case2.train(shuffled_train_data_array, shuffled_train_label_array)
case2.test(test_data_array, test_label_array)
case2.missclas_counter(train_data_array, train_label_array)
case2.missclas_counter(test_data_array, test_label_array)

Epoch 1 Train Error: 11843.7139875  Validation Error: 1311.3161406
Epoch 2 Train Error: 10008.4604964  Validation Error: 829.9063554
Epoch 3 Train Error: 6280.7240308  Validation Error: 525.7422314
Epoch 4 Train Error: 4594.8994374  Validation Error: 419.4115247
Epoch 5 Train Error: 3880.6033288  Validation Error: 352.4607461
Epoch 6 Train Error: 3411.8802061  Validation Error: 307.2390958
Epoch 7 Train Error: 3093.1379139  Validation Error: 273.6607031
Epoch 8 Train Error: 2824.4513831  Validation Error: 242.8961820
Epoch 9 Train Error: 2428.6836143  Validation Error: 190.7060538
Epoch 10 Train Error: 1710.2904863  Validation Error: 124.6120121
Epoch 11 Train Error: 1278.3105487  Validation Error: 96.8922664
Epoch 12 Train Error: 1093.5030782  Validation Error: 84.0486839
Epoch 13 Train Error: 984.8425285  Validation Error: 76.2381086
Epoch 14 Train Error: 905.7702791  Validation Error: 70.8940671
Epoch 15 Train Error: 843.7864081  Validation Error: 66.8017254
Epoch 16 Train Error: 79

In [20]:
case3.initilaze(300, 200, 0.001, 0, 100, 0)
case3.train(shuffled_train_data_array, shuffled_train_label_array)
case3.test(test_data_array, test_label_array)
case3.missclas_counter(train_data_array, train_label_array)
case3.missclas_counter(test_data_array, test_label_array)

Epoch 1 Train Error: 11801.2787581  Validation Error: 1259.7059390
Epoch 2 Train Error: 8792.2341528  Validation Error: 751.7007369
Epoch 3 Train Error: 5507.7701969  Validation Error: 475.8608447
Epoch 4 Train Error: 4276.5185293  Validation Error: 391.2865640
Epoch 5 Train Error: 3666.9153173  Validation Error: 333.6136702
Epoch 6 Train Error: 3253.9235895  Validation Error: 292.0604427
Epoch 7 Train Error: 2927.2381689  Validation Error: 254.5670520
Epoch 8 Train Error: 2431.8287733  Validation Error: 187.3431958
Epoch 9 Train Error: 1661.9754570  Validation Error: 124.1245168
Epoch 10 Train Error: 1278.9088626  Validation Error: 97.6366464
Epoch 11 Train Error: 1105.5446844  Validation Error: 84.3695882
Epoch 12 Train Error: 997.7122272  Validation Error: 76.2309468
Epoch 13 Train Error: 918.3533251  Validation Error: 70.5705032
Epoch 14 Train Error: 854.3223899  Validation Error: 66.2486196
Epoch 15 Train Error: 800.4436663  Validation Error: 62.7779388
Epoch 16 Train Error: 753.9

In [21]:
case4.initilaze(300, 200, 0.001, 0, 100, 0.9)
case4.train(shuffled_train_data_array, shuffled_train_label_array)
case4.test(test_data_array, test_label_array)
case4.missclas_counter(train_data_array, train_label_array)
case4.missclas_counter(test_data_array, test_label_array)

Epoch 1 Train Error: 11754.9128239  Validation Error: 1209.6727316
Epoch 2 Train Error: 8742.9076688  Validation Error: 756.4897551
Epoch 3 Train Error: 5550.8242739  Validation Error: 477.4812559
Epoch 4 Train Error: 4276.4704825  Validation Error: 390.0183204
Epoch 5 Train Error: 3654.3809265  Validation Error: 332.4709096
Epoch 6 Train Error: 3244.3924274  Validation Error: 291.3331237
Epoch 7 Train Error: 2920.1862867  Validation Error: 253.3747838
Epoch 8 Train Error: 2394.5327258  Validation Error: 180.8733556
Epoch 9 Train Error: 1617.5318316  Validation Error: 121.8188519
Epoch 10 Train Error: 1260.2084596  Validation Error: 96.6009864
Epoch 11 Train Error: 1094.3220626  Validation Error: 83.6826359
Epoch 12 Train Error: 990.0345477  Validation Error: 75.7598399
Epoch 13 Train Error: 913.3054476  Validation Error: 70.1424979
Epoch 14 Train Error: 851.0141272  Validation Error: 65.9294369
Epoch 15 Train Error: 799.3022482  Validation Error: 62.4606858
Epoch 16 Train Error: 754.6

In [22]:
case5.initilaze(300, 100, 0.01, 0, 100, 0)
case5.train(shuffled_train_data_array, shuffled_train_label_array)
case5.test(test_data_array, test_label_array)
case5.missclas_counter(train_data_array, train_label_array)
case5.missclas_counter(test_data_array, test_label_array)

Epoch 1 Train Error: 6304.4503390  Validation Error: 356.9728060
Epoch 2 Train Error: 2488.8412031  Validation Error: 148.6388657
Epoch 3 Train Error: 1474.5206109  Validation Error: 104.0818236
Epoch 4 Train Error: 1147.4149038  Validation Error: 69.2799703
Epoch 5 Train Error: 959.0329492  Validation Error: 70.0932927
Epoch 6 Train Error: 866.7278655  Validation Error: 57.2003016
Epoch 7 Train Error: 756.0890828  Validation Error: 55.8245615
Epoch 8 Train Error: 719.8664761  Validation Error: 56.0826655
Epoch 9 Train Error: 635.5399094  Validation Error: 53.6822105
Epoch 10 Train Error: 600.3064987  Validation Error: 51.1824764
Epoch 11 Train Error: 523.1849752  Validation Error: 45.8966552
Epoch 12 Train Error: 486.3176439  Validation Error: 49.8130659
Epoch 13 Train Error: 487.4978151  Validation Error: 45.4824267
Epoch 14 Train Error: 451.7394874  Validation Error: 45.6514039
Epoch 15 Train Error: 439.5369197  Validation Error: 44.7139710
Epoch 16 Train Error: 443.2505432  Validat

In [23]:
case6.initilaze(300, 100, 0.01, 0, 100, 0.9)
case6.train(shuffled_train_data_array, shuffled_train_label_array)
case6.test(test_data_array, test_label_array),
case6.missclas_counter(train_data_array, train_label_array)
case6.missclas_counter(test_data_array, test_label_array)

Epoch 1 Train Error: 6531.0447468  Validation Error: 365.0648510
Epoch 2 Train Error: 2555.6349028  Validation Error: 152.1769613
Epoch 3 Train Error: 1513.3907207  Validation Error: 93.6719196
Epoch 4 Train Error: 1173.7764143  Validation Error: 69.8836421
Epoch 5 Train Error: 986.5604545  Validation Error: 67.1705980
Epoch 6 Train Error: 874.5240738  Validation Error: 88.7810709
Epoch 7 Train Error: 777.9835585  Validation Error: 57.0808197
Epoch 8 Train Error: 750.4568745  Validation Error: 57.5970717
Epoch 9 Train Error: 603.4808548  Validation Error: 55.5407212
Epoch 10 Train Error: 572.3638381  Validation Error: 53.1753048
Epoch 11 Train Error: 521.4498472  Validation Error: 51.6487593
Epoch 12 Train Error: 496.2757539  Validation Error: 49.4493251
Epoch 13 Train Error: 517.6653494  Validation Error: 47.4907015
Epoch 14 Train Error: 463.7171051  Validation Error: 45.0851702
Epoch 15 Train Error: 453.2417135  Validation Error: 41.0204100
Epoch 16 Train Error: 421.0103974  Validati

In [24]:
case7.initilaze(300, 200, 0.01, 0, 100, 0)
case7.train(shuffled_train_data_array, shuffled_train_label_array)
case7.test(test_data_array, test_label_array)
case7.missclas_counter(train_data_array, train_label_array)
case7.missclas_counter(test_data_array, test_label_array)

Epoch 1 Train Error: 6117.6493454  Validation Error: 336.5034085
Epoch 2 Train Error: 2352.5712622  Validation Error: 146.7386505
Epoch 3 Train Error: 1468.8803636  Validation Error: 88.9493554
Epoch 4 Train Error: 1166.8741473  Validation Error: 72.3218546
Epoch 5 Train Error: 1018.1387010  Validation Error: 63.2447234
Epoch 6 Train Error: 879.5065723  Validation Error: 55.3814524
Epoch 7 Train Error: 789.3019413  Validation Error: 53.6981753
Epoch 8 Train Error: 696.7867850  Validation Error: 51.8356984
Epoch 9 Train Error: 681.9530465  Validation Error: 56.7317779
Epoch 10 Train Error: 651.4753627  Validation Error: 51.2539510
Epoch 11 Train Error: 566.8133958  Validation Error: 45.9045676
Epoch 12 Train Error: 595.7312268  Validation Error: 48.8585799
Epoch 13 Train Error: 508.0516687  Validation Error: 44.9962044
Epoch 14 Train Error: 464.7707800  Validation Error: 42.0343724
Epoch 15 Train Error: 436.3670997  Validation Error: 41.4884466
Epoch 16 Train Error: 400.9893389  Validat

In [25]:
case8.initilaze(300, 200, 0.01, 50, 100, 0.9)
case8.train(shuffled_train_data_array, shuffled_train_label_array)
case8.test(test_data_array, test_label_array)
case8.missclas_counter(train_data_array, train_label_array)
case8.missclas_counter(test_data_array, test_label_array)

Epoch 1 Train Error: 10856.4232291  Validation Error: 1102.7110343
Epoch 2 Train Error: 10035.4889318  Validation Error: 1102.7881266
Epoch 3 Train Error: 9857.3427152  Validation Error: 1175.4815378

Due to Validation Error Data: Early Stoping is happened at 3. Epoch.

Test Results are:
 In 2947 number of test data
 Accurarcy = %15.405497115710892
Miss clasification stats are:
Class 1:1386
Class 2:  1017
Class 3:  0
Class 4:  1203
Class 5:  1353
Class 6:  1343
Miss clasification stats are:
Class 1:496
Class 2:  437
Class 3:  0
Class 4:  491
Class 5:  532
Class 6:  537


In [26]:
case9.initilaze(300, 100, 0.001, 50, 100, 0)
case9.train(shuffled_train_data_array, shuffled_train_label_array)
case9.test(test_data_array, test_label_array)
case9.missclas_counter(train_data_array, train_label_array)
case9.missclas_counter(test_data_array, test_label_array)

Epoch 1 Train Error: 11838.8746794  Validation Error: 1306.4263561
Epoch 2 Train Error: 9866.8486862  Validation Error: 851.6642098
Epoch 3 Train Error: 6756.0009383  Validation Error: 592.3510552
Epoch 4 Train Error: 5349.9768687  Validation Error: 592.7371476
Epoch 5 Train Error: 5102.3381876  Validation Error: 541.4708249
Epoch 6 Train Error: 4652.3835950  Validation Error: 491.1211904
Epoch 7 Train Error: 3950.2309640  Validation Error: 445.6248672
Epoch 8 Train Error: 3699.6854543  Validation Error: 425.9934268
Epoch 9 Train Error: 3543.9424034  Validation Error: 385.2581028
Epoch 10 Train Error: 3844.4748407  Validation Error: 290.4507819
Epoch 11 Train Error: 3139.6941547  Validation Error: 233.1899356
Epoch 12 Train Error: 2219.3994981  Validation Error: 176.3202332
Epoch 13 Train Error: 1876.1866772  Validation Error: 137.6477169
Epoch 14 Train Error: 1629.6803391  Validation Error: 117.8443531
Epoch 15 Train Error: 1522.6547207  Validation Error: 120.6934085
Epoch 16 Train Er

In [27]:
case10.initilaze(300, 100, 0.001, 50, 100, 0.9)
case10.train(shuffled_train_data_array, shuffled_train_label_array)
case10.test(test_data_array, test_label_array)
case10.missclas_counter(train_data_array, train_label_array)
case10.missclas_counter(test_data_array, test_label_array)

Epoch 1 Train Error: 11839.1766145  Validation Error: 1306.6262636
Epoch 2 Train Error: 9992.7308032  Validation Error: 866.5929780
Epoch 3 Train Error: 6848.3138271  Validation Error: 601.0963431
Epoch 4 Train Error: 5447.3662433  Validation Error: 602.1369021
Epoch 5 Train Error: 5081.2678469  Validation Error: 576.0526292
Epoch 6 Train Error: 4781.2767751  Validation Error: 506.2729956
Epoch 7 Train Error: 4042.0086666  Validation Error: 451.4665939
Epoch 8 Train Error: 3780.8105039  Validation Error: 417.7273652
Epoch 9 Train Error: 3585.6481757  Validation Error: 403.3488647
Epoch 10 Train Error: 4012.8645823  Validation Error: 297.1204771
Epoch 11 Train Error: 2716.0766773  Validation Error: 243.1520715
Epoch 12 Train Error: 2592.3730276  Validation Error: 204.1039651
Epoch 13 Train Error: 1715.5315458  Validation Error: 136.8724839
Epoch 14 Train Error: 1747.9807252  Validation Error: 127.6312355
Epoch 15 Train Error: 1506.4803023  Validation Error: 107.0138709
Epoch 16 Train Er

In [28]:
case11.initilaze(300, 200, 0.001, 50, 100, 0)
case11.train(shuffled_train_data_array, shuffled_train_label_array)
case11.test(test_data_array, test_label_array)
case11.missclas_counter(train_data_array, train_label_array)
case11.missclas_counter(test_data_array, test_label_array)

Epoch 1 Train Error: 11816.7631168  Validation Error: 1279.7366018
Epoch 2 Train Error: 9226.1783311  Validation Error: 829.4074812
Epoch 3 Train Error: 6579.9760920  Validation Error: 579.4741018
Epoch 4 Train Error: 5381.0009073  Validation Error: 584.7905295
Epoch 5 Train Error: 4649.8155261  Validation Error: 523.8016906
Epoch 6 Train Error: 4553.3246361  Validation Error: 481.6415705
Epoch 7 Train Error: 3869.4856326  Validation Error: 426.4660990
Epoch 8 Train Error: 3576.4670350  Validation Error: 385.8427012
Epoch 9 Train Error: 3099.5889304  Validation Error: 308.8177030
Epoch 10 Train Error: 2388.4207100  Validation Error: 202.0624334
Epoch 11 Train Error: 2845.8323073  Validation Error: 163.0586385
Epoch 12 Train Error: 1730.2731132  Validation Error: 127.4229381
Epoch 13 Train Error: 1600.1004790  Validation Error: 114.8011389
Epoch 14 Train Error: 1491.7728729  Validation Error: 112.5151491
Epoch 15 Train Error: 1391.9112516  Validation Error: 113.0735772
Epoch 16 Train Er

In [29]:
case12.initilaze(300, 200, 0.001, 50, 100, 0.9)
case12.train(shuffled_train_data_array, shuffled_train_label_array)
case12.test(test_data_array, test_label_array)
case12.missclas_counter(train_data_array, train_label_array)
case12.missclas_counter(test_data_array, test_label_array)

Epoch 1 Train Error: 11677.5292416  Validation Error: 1149.4469242
Epoch 2 Train Error: 8722.9349496  Validation Error: 811.6172923
Epoch 3 Train Error: 6591.7581143  Validation Error: 615.4168411
Epoch 4 Train Error: 5095.2651613  Validation Error: 586.6399109
Epoch 5 Train Error: 4931.7239109  Validation Error: 504.0427464
Epoch 6 Train Error: 4082.2560826  Validation Error: 459.4268267
Epoch 7 Train Error: 3735.8730126  Validation Error: 423.6346178
Epoch 8 Train Error: 3494.9619061  Validation Error: 362.3941804
Epoch 9 Train Error: 3329.2146846  Validation Error: 235.8381458
Epoch 10 Train Error: 2275.0352667  Validation Error: 183.9508604
Epoch 11 Train Error: 1945.3522127  Validation Error: 144.9507509
Epoch 12 Train Error: 1696.0684044  Validation Error: 124.7776122
Epoch 13 Train Error: 1569.3682571  Validation Error: 123.0145687
Epoch 14 Train Error: 1440.5604619  Validation Error: 118.8168671
Epoch 15 Train Error: 1357.4767964  Validation Error: 114.2189048
Epoch 16 Train Er

In [30]:
case13.initilaze(300, 100, 0.01, 50, 100, 0)
case13.train(shuffled_train_data_array, shuffled_train_label_array)
case13.test(test_data_array, test_label_array)
case13.missclas_counter(train_data_array, train_label_array)
case13.missclas_counter(test_data_array, test_label_array)

Epoch 1 Train Error: 11707.2905080  Validation Error: 1165.2179325
Epoch 2 Train Error: 10849.8750578  Validation Error: 1296.2666230
Epoch 3 Train Error: 10607.6247270  Validation Error: 1103.1004604
Epoch 4 Train Error: 9777.2664769  Validation Error: 1097.0649387
Epoch 5 Train Error: 9698.5665170  Validation Error: 1091.1831945
Epoch 6 Train Error: 9727.5186981  Validation Error: 1059.5904799
Epoch 7 Train Error: 10465.1002553  Validation Error: 1102.6139042
Epoch 8 Train Error: 9701.9924006  Validation Error: 1039.4161038
Epoch 9 Train Error: 10338.2116551  Validation Error: 1090.7772235
Epoch 10 Train Error: 10143.0167329  Validation Error: 1196.2393255

Due to Validation Error Data: Early Stoping is happened at 10. Epoch.

Test Results are:
 In 2947 number of test data
 Accurarcy = %24.329826942653547
Miss clasification stats are:
Class 1:826
Class 2:  907
Class 3:  14
Class 4:  1203
Class 5:  1353
Class 6:  1343
Miss clasification stats are:
Class 1:225
Class 2:  429
Class 3:  1

In [31]:
case14.initilaze(300, 100, 0.01, 50, 100, 0.9)
case14.train(shuffled_train_data_array, shuffled_train_label_array)
case14.test(test_data_array, test_label_array)
case14.missclas_counter(train_data_array, train_label_array)
case14.missclas_counter(test_data_array, test_label_array)

Epoch 1 Train Error: 11203.6445844  Validation Error: 1312.6122280
Epoch 2 Train Error: 10362.8236182  Validation Error: 1056.3960170
Epoch 3 Train Error: 9965.0246013  Validation Error: 1305.0536831
Epoch 4 Train Error: 10242.6086310  Validation Error: 1077.4552667
Epoch 5 Train Error: 10305.7802407  Validation Error: 1093.0529943
Epoch 6 Train Error: 9936.7422448  Validation Error: 1133.9824526

Due to Validation Error Data: Early Stoping is happened at 6. Epoch.

Test Results are:
 In 2947 number of test data
 Accurarcy = %17.848659653885306
Miss clasification stats are:
Class 1:0
Class 2:  1056
Class 3:  796
Class 4:  1203
Class 5:  1353
Class 6:  1343
Miss clasification stats are:
Class 1:0
Class 2:  471
Class 3:  390
Class 4:  491
Class 5:  532
Class 6:  537


In [32]:
case15.initilaze(300, 200, 0.01, 0, 100, 0)
case15.train(shuffled_train_data_array, shuffled_train_label_array)
case15.test(test_data_array, test_label_array)
case15.missclas_counter(train_data_array, train_label_array)
case15.missclas_counter(test_data_array, test_label_array)

Epoch 1 Train Error: 5993.6499762  Validation Error: 356.6229930
Epoch 2 Train Error: 2238.0115796  Validation Error: 148.0599753
Epoch 3 Train Error: 1426.3743717  Validation Error: 86.4399996
Epoch 4 Train Error: 1143.9408851  Validation Error: 70.6085731
Epoch 5 Train Error: 944.5093320  Validation Error: 63.8406675
Epoch 6 Train Error: 828.0022450  Validation Error: 58.3157489
Epoch 7 Train Error: 778.7980417  Validation Error: 60.1186440
Epoch 8 Train Error: 662.3178510  Validation Error: 62.2945800

Due to Validation Error Data: Early Stoping is happened at 8. Epoch.

Test Results are:
 In 2947 number of test data
 Accurarcy = %92.26331862911435
Miss clasification stats are:
Class 1:8
Class 2:  0
Class 3:  0
Class 4:  172
Class 5:  30
Class 6:  0
Miss clasification stats are:
Class 1:26
Class 2:  14
Class 3:  39
Class 4:  131
Class 5:  17
Class 6:  1


In [33]:
case16.initilaze(300, 200, 0.01, 0, 100, 0.9)
case16.train(shuffled_train_data_array, shuffled_train_label_array)
case16.test(test_data_array, test_label_array)
case16.missclas_counter(train_data_array, train_label_array)
case16.missclas_counter(test_data_array, test_label_array)

Epoch 1 Train Error: 6076.2379934  Validation Error: 346.2657610
Epoch 2 Train Error: 2240.2007088  Validation Error: 139.8591350
Epoch 3 Train Error: 1418.7634445  Validation Error: 91.5891617
Epoch 4 Train Error: 1150.4889823  Validation Error: 69.1419466
Epoch 5 Train Error: 990.6937331  Validation Error: 62.1680698
Epoch 6 Train Error: 815.8256928  Validation Error: 57.0386618
Epoch 7 Train Error: 741.3219233  Validation Error: 55.1921285
Epoch 8 Train Error: 679.0302767  Validation Error: 52.7266007
Epoch 9 Train Error: 636.7614496  Validation Error: 48.3422774
Epoch 10 Train Error: 601.1081804  Validation Error: 53.2426914
Epoch 11 Train Error: 532.9897959  Validation Error: 46.5928489
Epoch 12 Train Error: 512.0397025  Validation Error: 48.1576867
Epoch 13 Train Error: 478.5198301  Validation Error: 45.6980664
Epoch 14 Train Error: 458.6179721  Validation Error: 40.5924812
Epoch 15 Train Error: 543.4058567  Validation Error: 45.0072630
Epoch 16 Train Error: 561.4658025  Validati